# Ex06 - Primeiros passos para a Transformada de Fourier

### Parte 1 - Gerando ondas senoidas 1D e 2D

Melhore o notebook sobre ondas senoidais ([link](08 Ondas senoidais.ipynb)), gerando outras senoides 2D e tentando interpretar o significado dos períodos $T_r$, $T_c$ e $T$, bem como das frequências normalizadas $u$ e $v$. Comece tentando gerar senoides 2D, onde $T_r$ ou $T_c$ é nulo. 

### Parte 1:
* $T_r$ e $T_c$ são os períodos vertical e horizontal, respectivamente, da senoide 2d.  
Períodos $T_r$ ou $T_c$ tendendo a zero significaria frequência tendendo ao infinito.

In [ ]:
theta = 45 * np.pi/180
# T_r = T/np.sin(theta)
T_r = 0.0009
T = T_r * np.sin(theta)
T_c = T/np.cos(theta)
H, W = 256, 256
s = (H, W)
v = H/T_r
u = W/T_c
print('T:', T)
print('T_r:', T_r)
print('T_c:', T_c)
print('v:', v)
print('u:', u)

f_ex1 = ia.cos(s, T, theta, 0)

In [ ]:
plt.imshow(f_ex1, cmap='gray')
plt.show()
plt.plot(f_ex1[0])
plt.show()
plt.plot(f_ex1[:,0])
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft2
import sys,os
ia898path = os.path.abspath('../../')
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

In [ ]:
H = 128
W = 256
s = (H,W)
T = 40
theta = 45 * np.pi/180
f = ia.cos(s, T, theta, 0)
ia.adshow(ia.normalize(f, [0,255]))

**OBS: estou usando a função adshow no lugar da implot pois percebi um efeito quadriculado na visualização (vejam abaixo).**

In [ ]:
plt.plot(f[0])
plt.show()

plt.plot(f[:,0])
plt.show()

In [ ]:
plt.imshow(f, cmap='gray')
plt.show()

### Parte 2 - Calculando a translação periódica

A próxima atividade consiste em entender as diferentes implementações da translação periódica. Procure comparar as 3 versões ([translação periódica](09 Translacao periodica.ipynb)). 

1. Primeiro, transforme a primeira implementação em uma função que recebe parâmetros para realizar a translação (rr, cc). 
2. Implemente a terceira forma indicada no link acima (usando slicing). 
3. Compare as 3 formas, em termos de resultado e de tempo. É provável que a última opção seja a mais eficiente de todas, pois somente utiliza fatiamento.

Um dos cuidados nas implementações é observar que os parâmetros da translação podem ser negativos ou ainda bem maiores que a largura ou altura da imagem.



**Fernando:** Implementação da função da primeira forma de translação

In [ ]:
def shift_image(img, r, c):
    
    f = np.array(img)
    H, W = f.shape
    dh,dw = (-r,-c)
    dhi = (-dh + H) % H
    dwi = (-dw + W) % W
    f2 = np.vstack((f, f))
    f4 = np.hstack((f2, f2))
    g = f4[dhi:dhi+H,dwi:dwi+W]
    return g

**Fernando:** Copiando a segunda forma do notebook:

In [ ]:
def ptrans2d(img,t):
    f = np.array(img)
    rr,cc = t
    H,W = f.shape
    
    r = rr%H
    c = cc%W
        
    g = np.empty_like(f)
    
    g[:r,:c] = f[H-r:H,W-c:W]
    g[:r,c:] = f[H-r:H,0:W-c]
    g[r:,:c] = f[0:H-r,W-c:W]
    g[r:,c:] = f[0:H-r,0:W-c]

    return g

In [ ]:
import matplotlib.image as mpimg

img = mpimg.imread('../data/cameraman.tif')
f1 = shift_image(img, 12, 128)
plt.imshow(f1, cmap='gray')
plt.show()

**Fernando:** Implementação da terceira forma por slicing.
* A parte comentada aloca o array g antes de atribuir os valores, mas essa implementação teve desempenho mais baixo.
* Me pareceu que a implementação ptrans2d já é por fatiamento.

In [ ]:
def shift_slice(img, rr, cc):
    
    f = np.array(img)
    r, c = f.shape
    if (rr < 0):
        r1 = r + rr
    else:
        r1 = rr
    if (cc < 0):
        c1 = c + cc
    else:
        c1 = cc
        
    g = np.concatenate(( np.concatenate((f[r1:,c1:], f[r1:,:c1]), axis=1) , \
                         np.concatenate((f[:r1, c1:], f[:r1, :c1]), axis=1) ))

#     g = np.zeros(f.shape)
#     g[:r-r1, :c-c1], g[:r-r1,c-c1:], g[r-r1:,:c-c1], g[r-r1:,c-c1:] = \
#     f[r1:,c1:],      f[r1:,:c1],     f[:r1, c1:],    f[:r1, :c1]

    return g

In [ ]:
img = mpimg.imread('../data/cameraman.tif')
f1 = shift_slice(img, 128, 128)
plt.imshow(f1, cmap='gray')
plt.show()

In [ ]:
img = mpimg.imread('../data/cameraman.tif')
f1 = ptrans2d(img, (50, 128))
plt.imshow(f1, cmap='gray')
plt.show()

Comparação de tempos:

In [ ]:
%timeit shift_image(img, 12, 128)

In [ ]:
%timeit ptrans2d(img, (12, 128))

In [ ]:
%timeit shift_slice(img, 12, 128)

### Parte 3 - Calculando a Transformada de Fourier de uma onda senoidal

Antes de continuar, leia sobre a Transformada Discreta de Fourier (DFT) e sobre a Transformada Rápida de Fourier (FFT). Além dos slides colocados no Google Calssroom, dê uma olhada em alguns links interessantes:

- [demo](http://bigwww.epfl.ch/demo/ip/demos/FFT/)
- [explicação intuitiva](https://betterexplained.com/articles/an-interactive-guide-to-the-fourier-transform/)

No exemplo abaixo, a partir da onda senoidal bidimensional gerada na parte 1, calculamos a Transformada de Fourier usando a função *fft2* do numpy. Para a visualização, é necessário fazer a translação periódica, para colocar a origem do espectro (0,0) no centro da imagem. Também é preciso usar o valor da magnitude do espectro resultante, já que o resultado da transformação é uma matriz complexa. E por fim, costumamos visualizar o log do resultado.

### Parte 3.1

In [ ]:
from numpy.fft import rfft2, fftshift
from numpy.fft import ifft2

faux = f.copy()
F = fft2(faux)
H,W = faux.shape

# sem log, sem transpose
fig = plt.figure(figsize=[15, 10])
fig.add_subplot(121)
plt.title('Sem log, sem transpose')
plt.imshow(ia.normalize(np.abs(F)), cmap='gray')
fig.add_subplot(122)
plt.title('Sem log, com transpose')
plt.imshow(shift_slice(ia.normalize(np.abs(F)),H//2,W//2), cmap='gray')
plt.show()

# sem log
fig = plt.figure(figsize=[15, 10])
fig.add_subplot(121)
plt.imshow((ia.ptrans(F,(H//2,W//2))+1).real, cmap='gray')
fig.add_subplot(122)
plt.imshow((ia.ptrans(F,(H//2,W//2))+1).imag, cmap='gray')
plt.show()


fig = plt.figure(figsize=[5, 5])
plt.imshow(ia.normalize((np.abs(ifft2(F)))), cmap='gray')
plt.show()

fig = plt.figure(figsize=[7, 7])
plt.imshow(ia.normalize(np.log(np.abs(shift_slice(F,H//2,W//2)+1)), [0,255]), cmap='gray')
plt.show()

fig = plt.figure(figsize=[7, 7])
F2 = ia.normalize(np.abs(shift_slice(F,H//2,W//2)))
plt.imshow(F2, cmap='gray')
plt.show()

**Fernando:** Pode-se perceber que a utilização do logaritmo na imagem faz com que a magnitude fique dentro de uma faixa menos acentuada. Desta forma, as diferenças entre os pixels de maior valor e os de menor valor fica menor.

In [ ]:
print(ia.normalize(np.abs(F[60:65,120:125])))
print(ia.normalize(np.log(np.abs(F[60:65,120:125])) + 1))

F.ravel()[517]

In [ ]:
plt.plot(np.log(np.abs(shift_slice(F,H//2,W//2)+1)))
plt.show()

1. Experimente visualizar a Transformada de Fourier de diversas formas: sem fazer a translação periódica, sem calcular o logaritmo. Observe as diferenças e comente.
2. Crie uma função auxiliar de visualização da Transformada de Fourier, assim como fizeram para visualização de múltiplas imagens, pois ela será bastante utilizada neste e nos próximos notebooks.
3. Explique porque o espectro de Fourier não aparece como apenas 2 pontos (senoide). Modifique a senoide *f* para obter apenas 2 pontos na Transformada de Fourier. (DICA: a imagem *f* que estamos buscando ainda será uma senoide bidimensional, porém com alguma propriedade que evite o efeito observado.) Uma outra maneira de tentar entender o que está causando o problema é criar o espectro que se deseja de forma sintética (uma imagem com 2 pontos equidistantes do centro e diametralmente opostos - dizemos que é um par de pontos complexos conjugados) e fazer a transformada inversa de Fourier (ifft2 do numpy) para obter a imagem da senoide buscada.

### Parte 3.3:
Aparentemente, o espectro não aparece como 2 pontos porque a imagem não é exatamente periódica. Pode-se perceber isso quando fazemos um stack das imagens senoides. As bordas não ficam alinhadas perfeitamente, o que causa o efeito.  
A forma que eu encontrei para que a imagem fique exatamente periódica é fazer com que a quantidade de ciclos dentro da imagem seja sempre inteira. Mas, para isso, o ângulo acaba sendo levemente afetado, já que eu faço uma arredondamento para obter $u$ e $v$ inteiros.

In [ ]:
theta = 45 * np.pi/180
T = 40
H = 128
W = 256

T_r = T/np.sin(theta)
T_c = T/np.cos(theta)

v = int(round(H/T_r))
u = int(round(W/T_c))

r, c = np.indices((H, W))
f_ex3 = np.cos(2*np.pi*( ((v/H)*r) + ((u/W)*c) ))
plt.imshow(f_ex3, cmap='gray')

print(T_r)
print(T_c)

print(v)
print(u)

print(H/T_r)
print(W/T_c)

print(W)
print(H)
f2 = np.hstack((f_ex3,f_ex3))
f3 = np.vstack( (f2,f2) )
fig = plt.figure(figsize=[10,10])
plt.imshow(f3, cmap='gray')
plt.show()

In [ ]:
faux = f_ex3
F = fft2(faux)
H,W = faux.shape
ia.adshow(ia.normalize(np.log(np.abs(shift_slice(F,H//2,W//2)+1))), [0,255])